In [1]:
import wikipedia
import codecs
import collections
import sys
import re
import nltk
from nltk.tokenize import WhitespaceTokenizer
from string import punctuation

In [2]:
def get_texts_for_lang(lang, n=10): # функция для скачивания статей из википедии
    wikipedia.set_lang(lang)
    wiki_content = []
    pages = wikipedia.random(n)
    for page_name in pages:
        try:
            page = wikipedia.page(page_name)
        except wikipedia.exceptions.WikipediaException:
            print('Skipping page {}'.format(page_name))
            continue

        wiki_content.append('{}\n{}'.format(page.title, page.content.replace('==', '')))

    return wiki_content

In [3]:
def wikitexts(): # скачиваем по 100 статей для каждого языка. Это может занять какое-то время (5-10 минут. как правило)
    wiki_texts = {}
    langs = ['es', 'uk', 'be', 'fr']
    for lang in langs: # испанский в википедии — это es, украинский — uk, а белорусский — be
        wiki_texts[lang] = get_texts_for_lang(lang, 100)
        print(lang, len(wiki_texts[lang]))
    return wiki_texts

In [4]:
wiki_texts = wikitexts()

E:\programs\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file E:\programs\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Skipping page La Aldea (desambiguación)
Skipping page YAT
Skipping page Maquilla
Skipping page Rose Hill (desambiguación)
Skipping page Diamond Lake
es 95
Skipping page Єлизаветіно
Skipping page 1482 (значення)
Skipping page Максютово (Стерлітамацький район)
Skipping page Аккурган
Skipping page Церква Різдва Христового
Skipping page Гутиря
Skipping page Сент-Обен
Skipping page Кассандра (значення)
Skipping page Вулиця Дворцова
Skipping page Сайджьо
Skipping page Йоффе
Skipping page Лопушне (Міжгірський район)
uk 88
Skipping page Сцяпанаўка
Skipping page Новік
Skipping page Уласавічы
Skipping page Сяркі
Skipping page Чэмпіянат Беларусі па грэка-рымскай барацьбе 2015
Skipping page Дрычын
Skipping page Вярхоўскі
Skipping page Вялікі Бор
Skipping page Арэналь
Skipping page Баханскі сельсавет
Skipping page Шарсні
be 89
Skipping page Schindler
Skipping page Le Premier Pas
Skipping page Lehane
Skipping page Les Casseurs
Skipping page Brody
fr 95


In [6]:
def tokens(file):  # находит токены
    exclude = set(punctuation + '0123456789'+'–—'+'«»')
    file = ''.join(ch for ch in file if ch not in exclude)
    tkns = WhitespaceTokenizer().tokenize(file.lower()) 
    return tkns

In [7]:
def freqdict(corpus):
    freqwords = []
    freqs = collections.defaultdict(lambda: 0)
    
    for article in corpus:
        article = tokens(article)
        for word in article:
            if '' != word:
                freqs[word] += 1
        
    for word in sorted(freqs, key=lambda w: freqs[w], reverse=True)[:50]:
        #print('{}\t{}'.format(freqs[word], word))
        freqwords.append(word)
    
    return freqwords

In [8]:
es_r = set(freqdict(wiki_texts['es']))

In [9]:
fr_r = set(freqdict(wiki_texts['fr']))

In [10]:
uk_r = set(freqdict(wiki_texts['uk']))

In [11]:
be_r = set(freqdict(wiki_texts['be']))

In [12]:
# множество из всех элементов, не принадлежащие другим
es = es_r.difference(fr_r, uk_r, be_r)
uk = uk_r.difference(fr_r, es_r, be_r)
be = be_r.difference(fr_r, uk_r, es_r)
fr = fr_r.difference(es_r, uk_r, be_r)

In [13]:
def probab(file, freqd):
    k = 0
    for word in file:
        if word in freqd:
            k += 1
    return k

In [14]:
def langdetect(file, es, uk, be, fr):
    c_es = probab(file, es)
    c_uk = probab(file, uk)
    c_be = probab(file, be)
    c_fr = probab(file, fr)
    c_all = c_fr + c_es + c_uk + c_be
    p_es = c_es/c_all
    p_uk = c_uk/c_all
    p_be = c_be/c_all
    p_fr = c_fr/c_all
    probability = max(p_es, p_uk, p_be, p_fr)
    if probability == p_es:
        return 'es'
    elif probability == p_uk:
        return 'uk'
    elif probability == p_be:
        return 'be'
    else:
        return 'fr'

In [15]:
langdetect(tokens(wiki_texts['uk'][4]), es, uk, be, fr)

'uk'

In [16]:
import numpy as np
from sklearn.metrics import accuracy_score

In [21]:
def accuracy_dict(lang):
    true = []
    pred = []
    for i in range(len(wiki_texts[lang])):
        true.append(lang)
        pred.append(langdetect(tokens(wiki_texts[lang][i]), es, uk, be, fr))
    return accuracy_score(true, pred)

In [24]:
accuracy_dict('es')

1.0

In [22]:
accuracy_dict('uk')

0.98863636363636365

In [23]:
accuracy_dict('be')

1.0

In [25]:
accuracy_dict('fr')

1.0

#### Второй метод: частотные символьные n-граммы

In [26]:
from itertools import islice, tee

def make_ngrams(text):
    N = 3 # задаем длину n-граммы
    ngrams = zip(*(islice(seq, index, None) for index, seq in enumerate(tee(text, N))))
    ngrams = [''.join(x) for x in ngrams]
    return ngrams

In [31]:
def freq_ngr(corpus):
    freqs_ngr = collections.defaultdict(lambda: 0)
    freqngrms = []

    for article in corpus:
        for ngram in make_ngrams(article.replace('\n', '').lower()):
            freqs_ngr[ngram] += 1

    for ngram in sorted(freqs_ngr, key=lambda n: freqs_ngr[n], reverse=True)[:50]:
       # print('{}\t{}'.format(freqs_ngr[ngram], ngram))
        freqngrms.append(ngram)
    
    return freqngrms

In [36]:
es_rn = set(freq_ngr(wiki_texts['es']))

In [37]:
uk_rn = set(freq_ngr(wiki_texts['uk']))

In [38]:
be_rn = set(freq_ngr(wiki_texts['be']))

In [39]:
fr_rn = set(freq_ngr(wiki_texts['fr']))

In [40]:
es_n = es_rn.difference(fr_rn, uk_rn, be_rn)
uk_n = uk_rn.difference(fr_rn, es_rn, be_rn)
be_n = be_rn.difference(fr_rn, uk_rn, es_rn)
fr_n = fr_rn.difference(es_rn, uk_rn, be_rn)

In [41]:
def accuracy_ngrms(lang):
    true = []
    pred = []
    for i in range(len(wiki_texts[lang])):
        true.append(lang)
        pred.append(langdetect(make_ngrams(wiki_texts[lang][i]), es_n, uk_n, be_n, fr_n))
    return accuracy_score(true, pred)

In [42]:
accuracy_ngrms('es')

0.98947368421052628

In [43]:
accuracy_ngrms('fr')

1.0

In [44]:
accuracy_ngrms('uk')

1.0

In [45]:
accuracy_ngrms('be')

1.0

В первом методе результат для украинского был хуже, чем у других. Во втором методе для испанского. Но разница не большая. Скорее всего дело в текстах.

In [46]:
# тестируем
wiki_texts_test = wikitexts()

E:\programs\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file E:\programs\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Skipping page Codificación
Skipping page Condado de Cleburne
Skipping page Avia
Skipping page Mount Pleasant (desambiguación)
Skipping page Bloque
Skipping page Yubo
es 94
Skipping page Побережне
Skipping page Ляхово (Кармаскалинський район)
Skipping page Келдерару
Skipping page Перейма
Skipping page Клименко Володимир Петрович
Skipping page Анан
Skipping page Вересоч
Skipping page Жебровський
Skipping page Балаклійка
Skipping page Хмарівка
uk 90
Skipping page Задворанцы
Skipping page Дарашкевіч
Skipping page Лужок
Skipping page Раман Навумавіч Мачульскі
Skipping page Крыштаф Зяновіч
be 95
Skipping page La Mort de Lucrèce
Skipping page Brocco
Skipping page Henry Larsen
Skipping page Expresso (homonymie)
Skipping page Leptothrix
Skipping page Cremonini
Skipping page Lac qui parle (homonymie)
Skipping page Traité de Saint-Maur
Skipping page Favier
fr 91


In [47]:
es_r = set(freqdict(wiki_texts_test['es']))
fr_r = set(freqdict(wiki_texts_test['fr']))
uk_r = set(freqdict(wiki_texts_test['uk']))
be_r = set(freqdict(wiki_texts_test['be']))

In [48]:
es = es_r.difference(fr_r, uk_r, be_r)
fr = fr_r.difference(es_r, uk_r, be_r)
uk = uk_r.difference(be_r, fr_r, es_r)
be = be_r.difference(fr_r, uk_r, es_r)

In [49]:
accuracy_dict('es')

1.0

In [50]:
accuracy_dict('fr')

1.0

In [51]:
accuracy_dict('uk')

0.98863636363636365

In [52]:
accuracy_dict('be')

1.0

In [53]:
be_rn = set(freq_ngr(wiki_texts_test['be']))
uk_rn = set(freq_ngr(wiki_texts_test['uk']))
fr_rn = set(freq_ngr(wiki_texts_test['fr']))
es_rn = set(freq_ngr(wiki_texts_test['es']))

In [55]:
es_n = es_rn.difference(be_rn, fr_rn, be_rn)
be_n = be_rn.difference(uk_rn, fr_rn, es_rn)
uk_n = uk_rn.difference(be_rn, fr_rn, es_rn)
fr_n = fr_rn.difference(uk_rn, be_rn, es_rn)

In [56]:
accuracy_ngrms('es')

0.98947368421052628

In [57]:
accuracy_ngrms('be')

0.9887640449438202

In [58]:
accuracy_ngrms('uk')

1.0

In [59]:
accuracy_ngrms('fr')

1.0

#### Отчет:
    -И в тренировочной выборке и в тестовой для украинского языка результат словарного метода был хуже, чем для остальных языков.
    -Также и в тестовой и в тренировочной выборках для испанского языка результат n-gram был хуже, чем для остальных. Распознавание белорусского языка с помощью n-gram в тестовой выборке оказалось хуже, чем в тренировочной 